<font size="+2" color="brown"><i><b><center>"With hard work, you can get fire out of a stone."</center></b></i></font>

<font size="+3" color=purple ><b> <center><u>Text Helper Functions</u></center></b></font>

**Version 4**  :  First Run <br>
**Version 6**  :  Added 9 new regex function required for text processing.<br>
**Version 7**  :  (Current) Added few functions and modified scripts.<br>
**Version 8**  :  *Loading...*

# Table of content
* [1. Objective](#1)   
* [2. Data](#2)
* [3. Basic Data Explorers](#3)
    - [3.1 Missing values](#3.1)
    - [3.2 Count values](#3.2)
    - [3.3 Unique values](#3.3)
    - [3.4 Duplicate values](#3.4)
    - [3.5 Stat](#3.5)
* [4. Regex Helpers](#4)
    - [4.1 URL](#4.1)
    - [4.2 Emoticons](#4.2)
    - [4.3 Email](#4.3)
    - [4.4 Hash](#4.4)
    - [4.5 Mention](#4.5)
    - [4.6 Number](#4.6)
    - [4.7 Phone Number](#4.7)
    - [4.8 Year](#4.8)
    - [4.9 Non Alphanumeric](#4.9)
    - [4.10 Punctuations](#4.10)
    - [4.11 Stopwords](#4.11)
    - [4.12 N-grams](#4.12)
    - [4.13 Repetitive Character](#4.13)
    - [4.14 Dollar](#4.14)
    - [4.15 Number-Greater](#4.15)
    - [4.16 Number- Lesser](#4.16)
    - [4.17 OR](#4.17)
    - [4.18 AND](#4.18)
    - [4.19 Dates](#4.19)
    - [4.20 Only Words](#4.20)
    - [4.21 Only Numbers](#4.21)
    - [4.22 Boundaries](#4.22)
    - [4.23 Search](#4.23)
    
* [5. End Notes](#5)
    

<font size="+3" color="blue"><b>1. Objective</b></font><br><a id="1"></a>

The aim of this kernel is to provide helper function for basic text processing.This functions can aid you to understand the data much better and perform EDA.
My major focus will be on apply regex on text but still i have mentioned few basic starter codes on the 3rd part of kernel.Rest all sections will deal with text preprocessing.The whole kernel can be useful for everyone especially **beginners**.

<font size="+1"><i>Readers,I am making you lazy to code but at same time I am helping you out.Do utilize this kernel for any of your text oriented competitions.</i></font><br><br>
    
<font size="+1" color=chocolate ><b>Please appreciate me through your Upvote.</b></font>

<font size="+3" color="blue"><b>2. Data</b></font><br><a id="2"></a>

I will be using data from [Twitter Sentiment Analysis](https://www.kaggle.com/c/tweet-sentiment-extraction/data) competiton

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import emoji

#Count vectorizer for N grams
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# Nltk for tekenize and stopwords
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

#Loading data
df=pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


<font size="+3" color="blue"><b>3. Basic Data Explorers</b></font><br> <a id="3"></a>

<font size="+2" color="indigo"><b>3.1 Missing values</b></font><br><a id="3.1"></a>

In [2]:
def missing_value_of_data(data):
    total=data.isnull().sum().sort_values(ascending=False)
    percentage=round(total/data.shape[0]*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])


In [3]:
missing_value_of_data(df)
#  Reason for 0 percentage value = Round of 1 divided by 27481 will be 0

,Total,Percentage
selected_text,1,0.0
text,1,0.0
sentiment,0,0.0
textID,0,0.0


We will drop NA values

In [4]:
df=df.dropna()

<font size="+2" color="indigo"><b>3.2 Count Values</b></font><br><a id="3.2"></a>

In [5]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [6]:
count_values_in_column(df,'sentiment')

,Total,Percentage
neutral,11117,40.45
positive,8582,31.23
negative,7781,28.32


<font size="+2" color="indigo"><b>3.3 Unique Values</b></font><br><a id="3.3"></a>

In [7]:
def unique_values_in_column(data,feature):
    unique_val=pd.Series(data.loc[:,feature].unique())
    return pd.concat([unique_val],axis=1,keys=['Unique Values'])


In [8]:
unique_values_in_column(df,'sentiment')

,Unique Values
0,neutral
1,negative
2,positive


<font size="+2" color="indigo"><b>3.4 Duplicate Values</b></font><br><a id="3.4"></a>

In [9]:
    
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [10]:
duplicated_values_data(df)

,Columns,Duplicate count
0,textID,0
1,text,0
2,selected_text,5017
3,sentiment,27477


<font size="+2" color="indigo"><b>3.5 Stat</b></font><br><a id="3.5"></a>

In [11]:
df.describe()

,textID,text,selected_text,sentiment
count,27480,27480,27480,27480
unique,27480,27480,22463,3
top,3773846bc3,I seriously need to find out why my laptop som...,good,neutral
freq,1,1,199,11117


Since these kind of basic python functions are pretty much well known to all python users.We dont need much focus here.Still I will elaborate this section in upcoming versions.

<font size="+3" color="blue"><b>4. Regex Helpers</b></font><br> 
<a id="4"></a>

Major RE functions

* **re.findall**   - Module is used to search for “all” occurrences that match a given pattern.
* **re.sub**       - Substitute the matched RE patter with given text
* **re.match**     - The match function is used to match the RE pattern to string with optional flags
* **re.search**    - This method takes a regular expression pattern and a string and searches for that pattern with the string.


We will be mostly using re.findall to detect patterns.

<font size="+2" color="indigo"><b>4.1 URL</b></font><br><a id="4.1"></a>

In [12]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [13]:
sentence="I love spending time at https://www.kaggle.com/"
find_url(sentence)

'https://www.kaggle.com/'

In [14]:
df['url']=df['text'].apply(lambda x:find_url(x))

<font size="+2" color="indigo"><b>4.2 Emoticons</b></font><br><a id="4.2"></a>

### Find and convert emoji to text

In [15]:
def find_emoji(text):
    emo_text=emoji.demojize(text)
    line=re.findall(r'\:(.*?)\:',emo_text)
    return line

In [16]:
sentence="I love ⚽ very much 😁"
find_emoji(sentence)

# Emoji cheat sheet - https://www.webfx.com/tools/emoji-cheat-sheet/
# Uniceode for all emoji : https://unicode.org/emoji/charts/full-emoji-list.html

['soccer_ball', 'beaming_face_with_smiling_eyes']

In [17]:
df['emoji']=df['text'].apply(lambda x: find_emoji(x))

### Remove Emoji from text

In [18]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)



In [19]:
sentence="Its all about \U0001F600 face"
print(sentence)
remove_emoji(sentence)

Its all about 😀 face


'Its all about  face'

In [20]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

<font size="+2" color="indigo"><b>4.3 Email</b></font><br><a id="4.3"></a>

In [21]:
def find_email(text):
    line = re.findall(r'[\w\.-]+@[\w\.-]+',str(text))
    return ",".join(line)

In [22]:
sentence="My gmail is abc99@gmail.com"
find_email(sentence)

'abc99@gmail.com'

In [23]:
df['email']=df['text'].apply(lambda x: find_email(x))

<font size="+2" color="indigo"><b>4.4 Hash</b></font><br><a id="4.4"></a>

This value is especially to denote trends in twitter.

In [24]:
def find_hash(text):
    line=re.findall(r'(?<=#)\w+',text)
    return " ".join(line)

In [25]:
sentence="#Corona is trending now" 
find_hash(sentence)

'Corona'

In [26]:
df['hash']=df['text'].apply(lambda x: find_hash(x))

<font size="+2" color="indigo"><b>4.5 Mention</b></font><br><a id="4.5"></a>

@ - Used to mention someone in tweets

In [27]:
def find_at(text):
    line=re.findall(r'(?<=@)\w+',text)
    return " ".join(line)

In [28]:
sentence="@David,can you help me out"
find_at(sentence)

'David'

In [29]:
df['at_mention']=df['text'].apply(lambda x: find_at(x))

<font size="+2" color="indigo"><b>4.6 Number</b></font><br><a id="4.6"></a>

In [30]:
def find_number(text):
    line=re.findall(r'[0-9]+',text)
    return " ".join(line)

In [31]:
sentence="2833047 people are affected by corona now"
find_number(sentence)

'2833047'

In [32]:
df['number']=df['text'].apply(lambda x: find_number(x))

<font size="+2" color="indigo"><b>4.7 Phone Number</b></font><br><a id="4.7"></a>

Indian Mobile numbers have ten digit.I will write that pattern below

In [33]:
def find_phone_number(text):
    line=re.findall(r"\b\d{10}\b",text)
    return "".join(line)

In [34]:
find_phone_number("9998887776 is a phone number of Mark from 210,North Avenue")

'9998887776'

In [35]:
df['phone_number']=df['text'].apply(lambda x: find_phone_number(x))

<font size="+2" color="indigo"><b>4.8 Year</b></font><br><a id="4.8"></a>

Extract year from 1940 till 2020

In [36]:
def find_year(text):
    line=re.findall(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",text)
    return line

In [37]:
sentence="India got independence on 1947."
find_year(sentence)

['1947']

In [38]:
df['year']=df['text'].apply(lambda x: find_year(x))

<font size="+2" color="indigo"><b>4.9 Non Alphanumeric</b></font><br><a id="4.9"></a>

In [39]:
def find_nonalp(text):
    line = re.findall("[^A-Za-z0-9 ]",text)
    return line

In [40]:
sentence="Twitter has lots of @ and # in posts.(general tweet)"
find_nonalp(sentence)

['@', '#', '.', '(', ')']

In [41]:
df['non_alp']=df['text'].apply(lambda x: find_nonalp(x))

<font size="+2" color="indigo"><b>4.10 Punctuations</b></font><br><a id="4.10"></a>

In [42]:
def find_punct(text):
    line = re.findall(r'[!"\$%&\'()*+,\-.\/:;=#@?\[\\\]^_`{|}~]*', text)
    string="".join(line)
    return list(string)

In [43]:
example="Corona virus have kiled #24506 confirmed cases now.#Corona is un(tolerable)"
print(find_punct(example))

['#', '.', '#', '(', ')']


In [44]:
df['punctuation']=df['text'].apply(lambda x : find_punct(x))

<font size="+2" color="indigo"><b>4.11 Stopwords</b></font><br><a id="4.11"></a>

In [45]:
def stop_word_fn(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(text) 
    non_stop_words = [w for w in word_tokens if not w in stop_words] 
    stop_words= [w for w in word_tokens if w in stop_words] 
    return stop_words

In [46]:
example_sent = "This is a sample sentence, showing off the stop words filtration."
stop_word_fn(example_sent)

['is', 'a', 'off', 'the']

In [47]:
df['stop_words']=df['text'].apply(lambda x : stop_word_fn(x))

<font size="+2" color="indigo"><b>4.12 N-grams</b></font><br><a id="4.12"></a>

In [48]:
def ngrams_top(corpus,ngram_range,n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english',ngram_range=ngram_range).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    total_list=words_freq[:n]
    df=pd.DataFrame(total_list,columns=['text','count'])
    return df

In [49]:
ngrams_top(df['text'],(1,1),n=10)

,text,count
0,just,2278
1,day,2115
2,good,1578
3,like,1353
4,http,1247
5,work,1150
6,today,1147
7,love,1145
8,going,1103
9,got,1085


In [50]:
ngrams_top(df['text'],(2,2),n=10)

,text,count
0,mother day,358
1,twitpic com,334
2,http twitpic,332
3,mothers day,279
4,happy mother,275
5,just got,219
6,happy mothers,199
7,http bit,180
8,bit ly,180
9,good morning,176


In [51]:
ngrams_top(df['text'],(3,3),n=10)

,text,count
0,http twitpic com,332
1,happy mother day,268
2,happy mothers day,195
3,http bit ly,180
4,http tinyurl com,166
5,http plurk com,109
6,star wars day,73
7,http blip fm,73
8,happy star wars,56
9,just got home,53


<font size="+2" color="indigo"><b>4.13 Repetitive Character</b></font><br><a id="4.13"></a>

In [52]:
def rep(text):
    grp = text.group(0)
    if len(grp) > 1:
        return grp[0:1] # can change the value here on repetition
    return grp
   
def unique_char(rep,sentence):
    convert = re.sub(r'(\w)\1+', rep, sentence) 
    return convert
 

In [53]:
sentence="heyyy this is loooong textttt"
unique_char(rep,sentence)

'hey this is long text'

In [54]:
df['unique_char']=df['text'].apply(lambda x : unique_char(rep,x))

<font size="+2" color="indigo"><b>4.14 Dollar</b></font><br><a id="4.14"></a>

In [55]:
def find_dollar(text):
    line=re.findall(r'\$\d+(?:\.\d+)?',text)
    return " ".join(line)

# \$ - dollar sign followed by
# \d+ one or more digits
# (?:\.\d+)? - decimal which is optional

In [56]:
sentence="this shirt costs $20.56"
find_dollar(sentence)

'$20.56'

In [57]:
df['dollar']=df['text'].apply(lambda x : find_dollar(x))

<font size="+2" color="indigo"><b>4.15 Number-Greater</b></font><br><a id="4.15"></a>

In [58]:
#Number greater than 930
def num_great(text): 
    line=re.findall(r'9[3-9][0-9]|[1-9]\d{3,}',text)
    return " ".join(line)

In [59]:
sentence="It is expected to be more than 935 corona death and 29974 observation cases across 29 states in india"
num_great(sentence)

'935 29974'

In [60]:
#Number greater than 930 (Just part of example)
df['num_great']=df['text'].apply(lambda x : num_great(x))

<font size="+2" color="indigo"><b>4.16 Number Lesser</b></font><br><a id="4.16"></a>

In [61]:
# Number less than 930
def num_less(text):
    only_num=[]
    for i in text.split():
        line=re.findall(r'^(9[0-2][0-0]|[1-8][0-9][0-9]|[1-9][0-9]|[0-9])$',i) # 5 500
        only_num.append(line)
        all_num=[",".join(x) for x in only_num if x != []]
    return " ".join(all_num)

In [62]:
sentence="There are some countries where less than 920 cases exist with 1100 observations"
num_less(sentence)

'920'

In [63]:
#Number greater than 930 (Just part of example)
df['num_less']=df['text'].apply(lambda x : num_less(x))

<font size="+2" color="indigo"><b>4.17 OR</b></font><br><a id="4.17"></a>

In [64]:
def or_cond(text):
    line=re.findall(r"sad|sorrow", text) 
    return " ".join(line)

In [65]:
or_cond("sad and sorrow displays emotions")

'sad sorrow'

In [66]:
# Looks for sorrow or sad word
df['sad_or_sorrow']=df['text'].apply(lambda x : or_cond(x))

<font size="+2" color="indigo"><b>4.18 AND</b></font><br><a id="4.18"></a>

In [67]:
def and_cond(text):
    line=re.findall(r'(?=.*do)(?=.*die).*', text) 
    return " ".join(line)

In [68]:
print("Both string present:",and_cond("do or die is a motivating phrase"))
print("Only one string present :",and_cond('die word is other side of emotion'))

Both string present: do or die is a motivating phrase
Only one string present : 


In [69]:
# Looks for do and die both else empty
df['do_and_die']=df['text'].apply(lambda x : and_cond(x))

<font size="+2" color="indigo"><b>4.19 Dates</b></font><br><a id="4.19"></a>

In [70]:
# mm-dd-yyyy format 
def find_dates(text):
    line=re.findall(r'\b(1[0-2]|0[1-9])/(3[01]|[12][0-9]|0[1-9])/([0-9]{4})\b',text)
    return line


In [71]:
sentence="Todays date is 04/28/2020 for format mm/dd/yyyy, not 28/04/2020"
find_dates(sentence)

[('04', '28', '2020')]

In [72]:
df['dates']=df['text'].apply(lambda x : find_dates(x))

<font size="+2" color="indigo"><b>4.20 Only Words</b></font><br><a id="4.20"></a>

In [73]:
def only_words(text):
    line=re.findall(r'\b[^\d\W]+\b', text)
    return " ".join(line)


In [74]:
sentence="the world population has grown from 1650 million to 6000 million"
only_words(sentence)

'the world population has grown from million to million'

In [75]:
df['only_words']=df['text'].apply(lambda x : only_words(x))

<font size="+2" color="indigo"><b>4.21 Only Numbers</b></font><br><a id="4.21"></a>

In [76]:
def only_numbers(text):
    line=re.findall(r'\b\d+\b', text)
    return " ".join(line)

In [77]:
sentence="the world population has grown from 1650 million to 6000 million"
only_numbers(sentence)

'1650 6000'

In [78]:
df['only_num']=df['text'].apply(lambda x : only_numbers(x))

<font size="+2" color="indigo"><b>4.22 Boundaries</b></font><br><a id="4.22"></a>

In [79]:
# Extracting word with boundary
def boundary(text):
    line=re.findall(r'\bneutral\b', text)
    return " ".join(line)

In [80]:
sentence="Most tweets are neutral in twitter"
boundary(sentence)

'neutral'

In [81]:
df['bound']=df['text'].apply(lambda x : boundary(x))

<font size="+2" color="indigo"><b>4.23 Search</b></font><a id="4.23"></a>

In [82]:
def search_string(text):
    return bool(re.search(r'day', text))

In [83]:
search_string("Happy Mothers day to all Moms")

True

In [84]:
df['search_day']=df['text'].apply(lambda x : search_string(x))

## *Data with new Features*

In [85]:
df.sample(10)
# We will see empty values too as most of text may not have related feature.You can filter and check.

,textID,text,selected_text,sentiment,url,emoji,email,hash,at_mention,number,...,dollar,num_great,num_less,sad_or_sorrow,do_and_die,dates,only_words,only_num,bound,search_day
15416,66095230ee,Spending QT with the hubby,QT,positive,,[],,,,,...,,,,,,[],Spending QT with the hubby,,,False
11094,b08f4b5a1b,Peeps! Me blackberry messenger is kwijt.. no ...,Peeps! Me blackberry messenger is kwijt.. no ...,neutral,,[],,,,,...,,,,,,[],Peeps Me blackberry messenger is kwijt no ping...,,,False
10269,0f9911e4c2,umm...why will it not let me change my backgro...,im twitter-challenged.,negative,,[],,,,,...,,,,,,[],umm why will it not let me change my backgroun...,,,False
2259,688e58880b,me and dee mack talking about your baby! She...,She`s almost all grown up!,positive,,[],,,,,...,,,,,,[],me and dee mack talking about your baby She s ...,,,False
2286,1871cab8f0,You`re right....I`m 29 and I`ve heard the so...,You`re right.,positive,,[],,,,29,...,,,29,,,[],You re right I m and I ve heard the song a tim...,29,,False
19581,3f36bc7958,heya 40 hour famine starts on may 22nd - are...,heya 40 hour famine starts on may 22nd - are ...,neutral,,[],,,,40 22,...,,,40,,,[],heya hour famine starts on may are you gonna t...,40,,False
3117,e4e206506b,"****, my photo is gone!","****, my photo is gone!",neutral,,[],,,,,...,,,,,,[],my photo is gone,,,False
2665,42385b0ddf,i won`t make u cry,i won`t make u cry,positive,,[],,,,,...,,,,,,[],i won t make u cry,,,False
6539,8dc0c34e5d,Worried about our Arabelle. Tweet please. Or ...,Worried,negative,,[],,,,,...,,,,,,[],Worried about our Arabelle Tweet please Or text,,,False
8196,a5e81f3773,Afternoon from office My cubicle has been mov...,! Mine is one of te few surviving 'good' system..,positive,,[],,,,,...,,,,,,[],Afternoon from office My cubicle has been move...,,,False


<font size="+3" color="blue"><b>5. End Notes</b></font><br> <a id="5"></a>

No worries.This is not the end of kernel.**I have updated the second version with few more functions (Now version 7 is live)**.Going forward i will add more helper functions in upcoming versions.So please <font size="+1" color="red"><b>Upvote</b></font> to keep them in your favourite list.

Thanks for your patience.


*Happy Kaggling!!!*.


<font size="+2" color="chocolate"><b><i>My other kernels</i></b></font><br><br>
<a id="5. End Note"></a>

1. **Twitter Sentiment Extraction :** I have made it with lots of EDAs and insights.I have used above mentioned helper function in it.      https://www.kaggle.com/raenish/tweet-sentiment-insight-eda

2. **Top COVID Kernels :** I made a dataset with all covid kernels data and presented a stat over it in a kernel.Hope you will enjoy it by looking how all those notebooks are widespread in kaggle community between different tier users.<br>
https://www.kaggle.com/raenish/top-covid-kernels-analysis
<br>

And yes,if you like those kernels,please give an appreciation to me  via an <font size="+1" color="red"><b>Upvote</b></font>.

<font size="+2" color="chocolate"><b>Reference</b></font><br>
* https://www.guru99.com/python-regular-expressions-complete-tutorial.html
* https://docs.python.org/3.4/library/re.html
* https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html#zz-1.9
* https://www.debuggex.com/cheatsheet/regex/python
* https://blog.finxter.com/python-regex-and-operator-tutorial-video/
* https://www.oreilly.com/library/view/regular-expressions-cookbook/9781449327453/ch04s04.html
* https://www.webfx.com/tools/emoji-cheat-sheet/

Version 8 :
<font size="+3" color="red"><b><i>Loading...</i></b></font><br>